In [13]:
using Plots, Interact, Krylov
gr()

Plots.GRBackend()

In [14]:
function more_sorensen(B, g, Δ)
    n = length(g)
    d = -B\g
    if norm(d) < Δ
        return d
    end
    
    λlow, λ = 0.0, 1.0
    
    d = -(B + λ*eye(n))\g
    while norm(d) > Δ
        λlow = λ
        λ *= 10.0
        d = -(B + λ*eye(n))\g
    end
    
    λhigh = λ
    λ = (λlow + λhigh)/2
    d = -(B + λ*eye(n))\g
    while abs(norm(d) - Δ) > 1e-6
        if norm(d) > Δ
            λlow = λ
        else
            λhigh = λ
        end
        λ = (λlow + λhigh)/2
        d = -(B + λ*eye(n))\g
    end
    
    return d
end

more_sorensen (generic function with 1 method)

In [15]:
t = linspace(0, 2pi, 100)
@manipulate for Δ = linspace(0.1, 2, 1000),
        B = Any[eye(2), [4.0 -1.0; -1.0 2.0],
        [16.0 5.0; 5.0 6.0], [6.0 5.0; 5.0 32.0]]
    g = -B*[1.2; 0.8] # Roubei
    m(d) = 0.5*dot(d, B*d) + dot(d,g)
    contour(linspace(-0.2, 2, 100), linspace(-0.2, 2, 100),
        (x,y)->m([x;y]), leg=false)
    plot!(Δ*cos(t), Δ*sin(t), c=:blue, l=:dash)
    
    # Gradiente
    plot!([0, -g[1]], [0, -g[2]], c=:blue)
    λ = dot(g,g)/dot(g,B*g)
    dC = -λ*g
    dN = -B\g
    scatter!([dC[1], dN[1]], [dC[2], dN[2]], c=:magenta, ms=4)
    plot!([dC[1], dN[1]], [dC[2], dN[2]], c=:magenta)
    
    d = more_sorensen(B, g, Δ)
    
    plot!([0, d[1]], [0, d[2]], c=:red)
    
    scg, _ = Krylov.cg(B, -g, radius=Δ)
    plot!([0, scg[1]], [0, scg[2]], c=:green, lw=2, l=:dash)
    
    xlims!(-0.2, 2)
    ylims!(-0.2, 2)
end

Interact.Slider{Float64}(Signal{Float64}(1.049049049049049, nactions=0),"Δ",1.049049049049049,linspace(0.1,2.0,1000),true)

Interact.Options{:ToggleButtons,Array{Float64,2}}(Signal{Array{Float64,2}}(2x2 Array{Float64,2}:
 1.0  0.0
 0.0  1.0, nactions=0),"B",2x2 Array{Float64,2}:
 1.0  0.0
 0.0  1.0,"[1.0 0.0\n 0.0 1.0]",Interact.OptionDict(Any["[1.0 0.0\n 0.0 1.0]","[4.0 -1.0\n -1.0 2.0]","[16.0 5.0\n 5.0 6.0]","[6.0 5.0\n 5.0 32.0]"],Dict{Any,Any}("[1.0 0.0\n 0.0 1.0]"=>2x2 Array{Float64,2}:
 1.0  0.0
 0.0  1.0,"[4.0 -1.0\n -1.0 2.0]"=>2x2 Array{Float64,2}:
  4.0  -1.0
 -1.0   2.0,"[6.0 5.0\n 5.0 32.0]"=>2x2 Array{Float64,2}:
 6.0   5.0
 5.0  32.0,"[16.0 5.0\n 5.0 6.0]"=>2x2 Array{Float64,2}:
 16.0  5.0
  5.0  6.0)),Any[],Any[])